<a href="https://colab.research.google.com/github/emmetorior/CN7030-/blob/main/ASSN_CN7050_FINAL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install findspark
!pip install --upgrade pyspark pyarrow pandas
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
import pandas as pd
import pyspark
import pyarrow as pa
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import col, explode, split, array, lit
from pyspark.ml.feature import CountVectorizer, IDF
import ast


Check Files Exist and Are Readable.

1.   List item
2.   List item


```
# This is formatted as code
```



In [ ]:
import os
parquet_validation_file = "/content/drive/MyDrive/Colab Notebooks/CN7050/validation-00000-of-00001.parquet"
print(os.path.exists(parquet_validation_file))

parquet_test_file = "/content/drive/MyDrive/Colab Notebooks/CN7050/test-00000-of-00001.parquet"
print(os.path.exists(parquet_test_file))

parquet_train_file = "/content/drive/MyDrive/Colab Notebooks/CN7050/train-00000-of-00001.parquet"
print(os.path.exists(parquet_train_file))

True
True
True


In [ ]:
spark = SparkSession.Builder().master("local[*]").getOrCreate()
spark.conf.set("spark.sql.parquet.mergeSchema", "false")
spark.conf.set("spark.sql.parquet.binaryAsString", "true")
#disable for now spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")

dfs_train = spark.read.format("parquet").option("mergeSchema", "false").load(parquet_train_file)
dfs_test = spark.read.format("parquet").option("mergeSchema", "false").load(parquet_test_file)
dfs_validation = spark.read.format("parquet").option("mergeSchema", "false").load(parquet_validation_file)

In [ ]:
dfs_train.show(5)
dfs_test.show(5)
dfs_validation.show(5)

+---------+---------+-----------+----------------+--------------------+--------------------+--------------------+
|       id|org_index|data_source|        industry|                text|              labels|         label_codes|
+---------+---------+-----------+----------------+--------------------+--------------------+--------------------+
|301972057|      600| Trustpilot|Price Comparison|My experience is ...|[[Staff support: ...|['staff-support.a...|
|301982453|      514|Google Play|         Banking|I love it so hand...|[[Company brand: ...|['company-brand.g...|
|301980653|      369|Google Play|    Ride Hailing|  Sometimes it takes|[[Company brand: ...|['company-brand.g...|
|301979991|      727|Apple Store|         Fashion|This is the worst...|[[Logistics rides...|['logistics-rides...|
|301984330|      549|Google Play|  Travel Booking|So easy & loads o...|[[Company brand: ...|['company-brand.g...|
+---------+---------+-----------+----------------+--------------------+-----------------

File was not loading. Implemented a Workaround by converting the pandas version

1.   List item
2.   List item

of the parquet file converted to CSV and read into spark to create a spark dataframe. Maybe delete this segment later as problem was resolved.  

In [ ]:
import pyarrow.parquet as pq

# Try with pyarrow first to validate the file
try:
    dfp_train = pq.read_table(parquet_train_file)
    dfp_test = pq.read_table(parquet_train_file)
    dfp_validation = pq.read_table(parquet_validation_file)
    print("File can be read with pyarrow")
except Exception as e:
    print(f"Pyarrow error: {e}")

File can be read with pyarrow


*italicized text*

In [ ]:
print("Training file row count:", dfs_train.count())
dfs_train.printSchema()
print("Testing file row count:", dfs_test.count())
dfs_test.printSchema()
print("Validation file row count:", dfs_validation.count())
dfs_validation.printSchema()

Training file row count: 7930
root
 |-- id: long (nullable = true)
 |-- org_index: long (nullable = true)
 |-- data_source: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- text: string (nullable = true)
 |-- labels: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- label_codes: string (nullable = true)

Testing file row count: 1587
root
 |-- id: long (nullable = true)
 |-- org_index: long (nullable = true)
 |-- data_source: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- text: string (nullable = true)
 |-- labels: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- label_codes: string (nullable = true)

Validation file row count: 1057
root
 |-- id: long (nullable = true)
 |-- org_index: long (nullable = true)
 |-- data_source: string (nullable = true)
 |-- industry: string (nullable = true

In [ ]:
#findspark.init()
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType



In [ ]:
#pd.read_parquet(parquet_file, engine='auto')
from pyspark.sql.functions import col

# Count nulls in each column - not working
#null_counts = test_df.select([col(c).isNull().sum().alias(c) for c in test_df.columns])
from pyspark.sql.functions import col, sum

null_counts = dfs_train.select([sum(col(c).isNull().cast("int")).alias(c) for c in dfs_train.columns])

null_counts.show()
# drop nulls
dfs_train_clean = dfs_train.dropna()
dfs_train_clean.show()


+---+---------+-----------+--------+----+------+-----------+
| id|org_index|data_source|industry|text|labels|label_codes|
+---+---------+-----------+--------+----+------+-----------+
|  0|        0|          0|       0|   0|     0|          0|
+---+---------+-----------+--------+----+------+-----------+

+---------+---------+-----------+----------------+--------------------+--------------------+--------------------+
|       id|org_index|data_source|        industry|                text|              labels|         label_codes|
+---------+---------+-----------+----------------+--------------------+--------------------+--------------------+
|301972057|      600| Trustpilot|Price Comparison|My experience is ...|[[Staff support: ...|['staff-support.a...|
|301982453|      514|Google Play|         Banking|I love it so hand...|[[Company brand: ...|['company-brand.g...|
|301980653|      369|Google Play|    Ride Hailing|  Sometimes it takes|[[Company brand: ...|['company-brand.g...|
|301979991

In [ ]:
#
# Initialize Spark session
#spark = SparkSession.builder.appName("ABSA_Project").getOrCreate()

# Assuming the dataframe is already loaded as 'df'
# If not, you can load it like this:
# df = spark.read.csv("export_45.csv", header=True, inferSchema=True)

# Display the schema to understand the data structure
#dfp_train.printSchema()
# Let's extract the categories from label_codes
# First, we need to convert the string representation of label codes to actual lists
# We'll use a UDF (User Defined Function) for this

# Define a UDF to extract just the category part from each label code
@udf(returnType=ArrayType(StringType()))
def extract_categories(label_codes):
    # Convert string representation to list
    if isinstance(label_codes, str):
        codes = ast.literal_eval(label_codes)
        # Extract category part (remove the sentiment indicator at the end)
        categories = [code.rsplit('.', 1)[0] for code in codes]
        return categories
    return []

#dfs_train = spark.createDataFrame(dfp_train)     #pandas df has no ... withColumn so we need to convert it - Oops it's not a pandas df, it's a pyarrow Table that needs to be first converted to a pandas df
#dfp_train = dfp_train.to_pandas()
#dfs_train = spark.createDataFrame(dfp_train)
##It seems that the to_pandas method is not available for Spark DataFrames -  use the collect method to convert the Spark DataFrame to a Pandas DataFrame:

dfp_train = spark.read.parquet(parquet_train_file)
dfp_train = dfp_train.collect()
dfs_train = spark.createDataFrame(dfp_train)


In [ ]:
df_with_categories = dfs_train.withColumn("categories", extract_categories(col("label_codes")))
# Create a list of all unique categories from the dataset
df_with_categories.printSchema()
df_with_categories.show(5)


root
 |-- id: long (nullable = true)
 |-- org_index: long (nullable = true)
 |-- data_source: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- text: string (nullable = true)
 |-- labels: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- label_codes: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---------+---------+-----------+----------------+--------------------+--------------------+--------------------+--------------------+
|       id|org_index|data_source|        industry|                text|              labels|         label_codes|          categories|
+---------+---------+-----------+----------------+--------------------+--------------------+--------------------+--------------------+
|301972057|      600| Trustpilot|Price Comparison|My experience is ...|[[Staff support: ...|['staff-support.a...|[staff-support.at

In [ ]:
all_categories = df_with_categories.select(explode("categories")).distinct().collect()
all_categories = [row[0] for row in all_categories]

In [ ]:
for cat in all_categories:
  print(cat + '\n')

value.price-value-for-money

company-brand.general-satisfaction

staff-support.attitude-of-staff

staff-support.email

company-brand.reviews

online-experience.app-website

purchase-booking-experience.ease-of-use

company-brand.competitor

value.discounts-promotions

logistics-rides.speed

account-management.account-access

staff-support.phone



In [ ]:
# For each category, create a binary indicator column (1 if present, 0 if not)
for cat in all_categories:
    df_with_categories = df_with_categories.withColumn(
        f"has_{cat.replace('-', '_').replace('.', '_')}",
        array_contains(col("categories"), cat).cast("integer")
    )


In [ ]:
df_with_categories.show()


+---------+---------+-----------+----------------+--------------------+--------------------+--------------------+--------------------+-------------------------------+--------------------------------------+-----------------------------------+-----------------------+-------------------------+---------------------------------+-------------------------------------------+----------------------------+------------------------------+-------------------------+-------------------------------------+-----------------------+
|       id|org_index|data_source|        industry|                text|              labels|         label_codes|          categories|has_value_price_value_for_money|has_company_brand_general_satisfaction|has_staff_support_attitude_of_staff|has_staff_support_email|has_company_brand_reviews|has_online_experience_app_website|has_purchase_booking_experience_ease_of_use|has_company_brand_competitor|has_value_discounts_promotions|has_logistics_rides_speed|has_account_management_acco

In [ ]:
df_with_categories['categories'].values[0]

Column<'categories[values][0]'>

In [ ]:

# Select relevant columns for the ACD task
feature_cols = [f"has_{category.replace('-', '_').replace('.', '_')}" for category in all_categories]
acd_df = df_with_categories.select("id", "text", *feature_cols)

# Convert text to features using TF-IDF
# First, tokenize the text
from pyspark.ml.feature import Tokenizer, StopWordsRemover

tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(acd_df)

# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filtered_data = remover.transform(wordsData)

# Convert words to term frequency features
cv = CountVectorizer(inputCol="filtered_words", outputCol="tf", minDF=2.0)
cv_model = cv.fit(filtered_data)
tf_data = cv_model.transform(filtered_data)

# Convert term frequency features to TF-IDF
idf = IDF(inputCol="tf", outputCol="features")
idf_model = idf.fit(tf_data)
tfidf_data = idf_model.transform(tf_data)

# Final dataset ready for modeling
final_acd_df = tfidf_data.select("id", "features", *feature_cols)

# Split the data into training and testing sets
train_data, test_data = final_acd_df.randomSplit([0.8, 0.2], seed=42)

# Cache the datasets for faster processing
train_data.cache()
test_data.cache()

# Now you can proceed with training your models

DataFrame[id: bigint, features: vector, has_value_price_value_for_money: int, has_company_brand_general_satisfaction: int, has_staff_support_attitude_of_staff: int, has_staff_support_email: int, has_company_brand_reviews: int, has_online_experience_app_website: int, has_purchase_booking_experience_ease_of_use: int, has_company_brand_competitor: int, has_value_discounts_promotions: int, has_logistics_rides_speed: int, has_account_management_account_access: int, has_staff_support_phone: int]

In [ ]:
#Since you're working with text data, you'll need to extract relevant features from each document. Some common techniques include:
#Tokenization (splitting text into individual words or tokens)
#Stopword removal (removing common words like "the", "and", etc.)
#Stemming or Lemmatization (reducing words to their base form)
#Part-of-speech tagging (identifying grammatical categories of each word)
# Create a SparkSession with your test/train/validate data sets
#spark = SparkSession.builder.appName("Aspect-Based Sentiment Analysis").getOrCreate()

# Load the training and validate datasets into DataFrames
#train_df = spark.read.csv("path/to/train/dataset", header=True, inferSchema=True)
#val_df = spark.read.csv("path/to/val/dataset", header=True, inferSchema=True)

# Define a function to extract features from each column in the dataset
def feature_extraction(df):
    # Tokenization and stopword removal
    df = df.select(
        explode(col("text").cast("string")).alias("tokens"),
        explode(col("sentiment").cast("string")).alias("sentiment")
    )

    # Stemming or Lemmatization
    df = df.withColumn(
        "stemmed_tokens",
        df.tokens.map(lambda x: x.lower() if x.isnumeric() else x)  # Remove numbers and convert to lowercase
    )

    # Part-of-speech tagging (optional)
    #df = df.withColumn("pos_tags", explode(col("word").cast("string")).map(lambda x: {"POS": "NNP"} if x.isnumeric() else {"POS": "NOUN"})  # Example

    return df

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(acd_df)

# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filtered_data = remover.transform(wordsData)

# Convert words to term frequency features
cv = CountVectorizer(inputCol="filtered_words", outputCol="tf", minDF=2.0)
cv_model = cv.fit(filtered_data)
tf_data = cv_model.transform(filtered_data)

# Convert term frequency features to TF-IDF
idf = IDF(inputCol="tf", outputCol="features")
idf_model = idf.fit(tf_data)
tfidf_data = idf_model.transform(tf_data)

# Final dataset ready for modeling
final_acd_df = tfidf_data.select("id", "features", *feature_cols)

# Split the data into training and testing sets
train_data, test_data = final_acd_df.randomSplit([0.8, 0.2], seed=42)

# Cache the datasets for faster processing
train_data.cache()
test_data.cache()

DataFrame[id: bigint, features: vector, has_value_price_value_for_money: int, has_company_brand_general_satisfaction: int, has_staff_support_attitude_of_staff: int, has_staff_support_email: int, has_company_brand_reviews: int, has_online_experience_app_website: int, has_purchase_booking_experience_ease_of_use: int, has_company_brand_competitor: int, has_value_discounts_promotions: int, has_logistics_rides_speed: int, has_account_management_account_access: int, has_staff_support_phone: int]

In [ ]:
train_data.show(10)

+---------+--------------------+-------------------------------+--------------------------------------+-----------------------------------+-----------------------+-------------------------+---------------------------------+-------------------------------------------+----------------------------+------------------------------+-------------------------+-------------------------------------+-----------------------+
|       id|            features|has_value_price_value_for_money|has_company_brand_general_satisfaction|has_staff_support_attitude_of_staff|has_staff_support_email|has_company_brand_reviews|has_online_experience_app_website|has_purchase_booking_experience_ease_of_use|has_company_brand_competitor|has_value_discounts_promotions|has_logistics_rides_speed|has_account_management_account_access|has_staff_support_phone|
+---------+--------------------+-------------------------------+--------------------------------------+-----------------------------------+-----------------------+-----

In [ ]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Function to train and evaluate an SVM model for one category
def train_svm_for_category(category, train_data, test_data):
    # Prepare label column name
    label_col = f"has_{category.replace('-', '_').replace('.', '_')}"

    # Create SVM model
    svm = LinearSVC(featuresCol="features", labelCol=label_col, maxIter=10)

    # Train the model
    model = svm.fit(train_data)

    # Make predictions
    predictions = model.transform(test_data)

    # For binary classification metrics (precision, recall)
    binary_evaluator = BinaryClassificationEvaluator(
        labelCol=label_col, rawPredictionCol="rawPrediction")

    # For F1 score (use MulticlassClassificationEvaluator)
    multi_evaluator = MulticlassClassificationEvaluator(
        labelCol=label_col, predictionCol="prediction", metricName="f1")

    # Calculate AUC (Area Under ROC)
    auc = binary_evaluator.evaluate(predictions)

    # Calculate F1
    f1 = multi_evaluator.evaluate(predictions)

    # Calculate precision and recall manually
    # First, get true positives, false positives, true negatives, false negatives
    tp = predictions.filter((col(label_col) == 1) & (col("prediction") == 1)).count()
    fp = predictions.filter((col(label_col) == 0) & (col("prediction") == 1)).count()
    fn = predictions.filter((col(label_col) == 1) & (col("prediction") == 0)).count()

    # Calculate precision and recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return {
        "category": category,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc
    }

In [ ]:
# Convert Spark DataFrame to Pandas for deep learning
# Collect the data (be careful with large datasets)
train_pandas = train_data.toPandas()
test_pandas = test_data.toPandas()

# We need to convert the sparse vector features to numpy arrays
from pyspark.ml.linalg import SparseVector

def sparse_to_array(sparse_vector):
    return sparse_vector.toArray() if isinstance(sparse_vector, SparseVector) else sparse_vector

# Convert features to numpy arrays
train_pandas['features_array'] = train_pandas['features'].apply(sparse_to_array)
test_pandas['features_array'] = test_pandas['features'].apply(sparse_to_array)

# Prepare X (features) and y (labels) for training
X_train = np.stack(train_pandas['features_array'].values)
X_test = np.stack(test_pandas['features_array'].values)

# Get all label columns
label_cols = [col for col in train_pandas.columns if col.startswith('has_')]
y_train = train_pandas[label_cols].values
y_test = test_pandas[label_cols].values

# Now build a deep learning model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(label_cols), activation='sigmoid')
])

# Compile model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Train model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32
)

# Evaluate on test set
results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

# Make predictions
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate F1 score manually
from sklearn.metrics import f1_score, precision_score, recall_score
precision = precision_score(y_test, y_pred_binary, average='micro')
recall = recall_score(y_test, y_pred_binary, average='micro')
f1 = f1_score(y_test, y_pred_binary, average='micro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.3194 - loss: 0.5167 - precision: 0.3219 - recall: 0.3179 - val_accuracy: 0.4793 - val_loss: 0.2767 - val_precision: 0.6927 - val_recall: 0.3246
Epoch 2/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5241 - loss: 0.2711 - precision: 0.7343 - recall: 0.4372 - val_accuracy: 0.5863 - val_loss: 0.2168 - val_precision: 0.7719 - val_recall: 0.5384
Epoch 3/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.6367 - loss: 0.1811 - precision: 0.8423 - recall: 0.6670 - val_accuracy: 0.6089 - val_loss: 0.1962 - val_precision: 0.7821 - val_recall: 0.6212
Epoch 4/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.6494 - loss: 0.1396 - precision: 0.8751 - recall: 0.7651 - val_accuracy: 0.6214 - val_loss: 0.1939 - val_precision: 0.7943 - val_recall: 0.6337
Epoch 5/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.6639 - loss: 0.1130 - precision: 0.9028 - recall: 0.8231 - val_accuracy: 0.6315 - val_los

In [ ]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler


# For simplicity, let's implement a binary classification model for each category
# In practice, you might want to use a multi-label approach

# Function to train and evaluate an SVM model for one category
def train_svm_for_category(category, train_data, test_data):
    # Prepare label column name
    label_col = f"has_{category.replace('-', '_').replace('.', '_')}"

    # Create SVM model
    svm = LinearSVC(featuresCol="features", labelCol=label_col, maxIter=10)

    # Train the model
    model = svm.fit(train_data)

    # Make predictions
    predictions = model.transform(test_data)

    # Evaluate model
   # evaluator = MulticlassClassificationEvaluator(
   #     labelCol=label_col, predictionCol="prediction", metricName="f1")
   # f1 = evaluator.evaluate(predictions)
    #
    evaluator = MulticlassClassificationEvaluator(labelCol=label_col,
                                              predictionCol="prediction",
                                              metricName="weightedPrecision")
precision = evaluator.evaluate(predictions)
print("Weighted Precision:", precision)
    evaluator.setMetricName("precision")
    precision = evaluator.evaluate(predictions)

    evaluator.setMetricName("recall")
    recall = evaluator.evaluate(predictions)

    return {"category": category, "precision": precision, "recall": recall, "f1": f1}

# Train models for all categories
svm_results = []
for category in all_categories:
    result = train_svm_for_category(category, train_data, test_data)
    svm_results.append(result)

# Print results
svm_results_df = spark.createDataFrame(svm_results)
svm_results_df.show()

IndentationError: unexpected indent (<ipython-input-23-3eaf62c79b90>, line 34)

In [ ]:
print("Training SVM models for all categories...")
svm_results = []
for category in all_categories:
    try:
        result = train_svm_for_category(category, train_data, test_data)
        svm_results.append(result)
        print(f"Completed category: {category}")
    except Exception as e:
        print(f"Error processing category {category}: {str(e)}")

# Create a dataframe with results
svm_results_df = spark.createDataFrame(svm_results)
print("SVM Results:")
svm_results_df.show()

# Calculate average metrics
avg_precision = np.mean([r["precision"] for r in svm_results])
avg_recall = np.mean([r["recall"] for r in svm_results])
avg_f1 = np.mean([r["f1"] for r in svm_results])

print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1: {avg_f1:.4f}")

# Convert to pandas DataFrame for deep learning model
# We'll process a subset of records to handle memory constraints
# For a real model, you might want to use a data loader or mini-batches
print("Preparing data for deep learning model...")
train_pandas = train_data.limit(1000).toPandas()  # Limit to 1000 samples for demonstration
test_pandas = test_data.limit(200).toPandas()

# Function to convert sparse vector to array
def sparse_to_array(v):
    if hasattr(v, 'toArray'):
        return v.toArray()
    return v

# Apply conversion function
train_pandas['features_array'] = train_pandas['features'].apply(sparse_to_array)
test_pandas['features_array'] = test_pandas['features'].apply(sparse_to_array)

# Stack features into numpy arrays
X_train = np.stack(train_pandas['features_array'].values)
X_test = np.stack(test_pandas['features_array'].values)

# Extract labels
y_train = train_pandas[feature_cols].values
y_test = test_pandas[feature_cols].values

print("Data preparation for deep learning complete!")

In [ ]:
# Train models for all categories
svm_results = []
for category in all_categories:
   # result = train_svm_for_category(category, train_data, test_data)
    result = train_svm_for_category(category, train_data, test_data)
    svm_results.append(result)

# Print results
svm_results_df = spark.createDataFrame(svm_results)
svm_results_df.show()

In [ ]:
# Identify categorical columns
categorical_cols = [c for (c, dtype) in dfs_train_clean.dtypes if dtype == 'string']

# Fill missing categorical values
for col_name in categorical_cols:
    dfs_train_clean = dfs_train_clean.fillna({col_name: "Unknown"})

# Verify nulls are removed
dfs_train_clean.select([sum(col(c).isNull().cast("int")).alias(c) for c in dfs_train_clean.columns]).show()

In [ ]:
from pyspark.sql.functions import col, mean

# Fill numerical columns with mean
numeric_cols = [c for (c, dtype) in dfs_train_clean.dtypes if dtype in ('int', 'double', 'float')]
for col_name in numeric_cols:
    mean_value = test_df.select(mean(col(col_name))).collect()[0][0]
    test_df = test_df.fillna({col_name: mean_value})
    train_df = train_df.fillna({col_name: mean_value})
    validation_df = validation_df.fillna({col_name: mean_value})

# Fill categorical columns with "Unknown"
categorical_cols = [c for (c, dtype) in dfs_train_clean.dtypes if dtype == 'string']
for col_name in categorical_cols:
    test_df = test_df.fillna({col_name: "Unknown"})
    train_df = train_df.fillna({col_name: "Unknown"})
    validation_df = validation_df.fillna({col_name: "Unknown"})

# Confirm there are no nulls left
test_df.show(5)

from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=col_name, outputCol=col_name + "_index").fit(train_df) for col_name in categorical_cols]
for indexer in indexers:
    train_df = indexer.transform(train_df)
    test_df = indexer.transform(test_df)
    validation_df = indexer.transform(validation_df)

# Drop original categorical columns (optional)
train_df = train_df.drop(*categorical_cols)
test_df = test_df.drop(*categorical_cols)
validation_df = validation_df.drop(*categorical_cols)

train_df.show(5)


In [ ]:
import pyarrow.parquet as pq

# Set the thrift string size limit to 1024 bytes
thrift_string_size_limit = 1024

# Set the thrift container size limit to 1048576 bytes (1MB)
thrift_container_size_limit = 1048576

# Read the parquet file
table = pq.read_table(parquet_train_file, thrift_string_size_limit=thrift_string_size_limit, thrift_container_size_limit=thrift_container_size_limit)

In [ ]:
#dfp_train = pd.read_parquet(parquet_train_file, engine="pyarrow")
#dfp_train.to_csv("converted_train.csv", index=False)

#dfs_train_converted = spark.read.csv("converted_train.csv", header=True, inferSchema=True)
#dfs_train_converted.show(5)